In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import glob
import pickle
import helper_functions_06b as hf

In [2]:
DATE = ['20230302']#, '20230305']
SESSION = ['SM_Lek1']#, 'SE_Lek1']
DRONE = ['P1D1', 'P1D2', 'P2D3', 'P2D4', 'P3D5', 'P3D6']

tracking_directory = '/Volumes/EAS_shared/blackbuck/working/processed/Field_Recording_2023/TestRegistration'

In [3]:
# Thresholds for translation, rotation, and scale
thresholds = {
    'T_translation': 30,  # e.g., 30 pixels
    'T_rotation': 20,     # e.g., 20 degrees
    'T_scale': 0.1         # e.g., 10% change
}

In [4]:
for date in DATE:
    for session in SESSION:
        for drone in DRONE:
            print(date, session, drone)
            
            internal_path = tracking_directory + '/' + date + '/' + session + '/' + drone
            files = glob.glob(internal_path + '/' + date + '_' + session + '_' + drone + '*_tracked.csv')
            
            for file in sorted(files):
                df = pd.read_csv(file)
                
                df['x'] = df['bb_left'] + df['bb_width']/2
                df['y'] = df['bb_top'] + df['bb_height']/2
                df['idx'] = df['track_id']

                df = df.loc[:,['frame', 'x', 'y', 'idx']]
                
                # Step 1: Identify anchor frames
                anchor_frames = hf.identify_anchor_frames(df, thresholds)

                # Step 2: Find the best anchor for each frame
                df = hf.find_best_anchor_for_each_frame_parallel(df, anchor_frames)

                # Step 3: Apply homography using the best anchor for each frame
                df, homography_matrices, rms_errors = hf.apply_homography_to_best_anchor(df)
                
                anchor_filename = os.path.basename(file).replace('_YOLO_tracked.csv', '_Anchored.csv')
                pickle_filename = os.path.basename(file).replace('_YOLO_tracked.csv', '_homographies.pkl')
                df.to_csv(os.path.join(os.path.dirname(file), anchor_filename), index=False, mode='w')
                with open(os.path.join(os.path.dirname(file), pickle_filename), 'wb') as f:
                    pickle.dump(homography_matrices, f)

20230302 SM_Lek1 P1D1
Initial identification of anchors takes  15.040010929107666
Finding the best anchors took 285.0100779533386
Applying homography to the best anchors takes  153.03230714797974
Initial identification of anchors takes  15.026885032653809
Finding the best anchors took 157.0563199520111
Applying homography to the best anchors takes  153.33962202072144
20230302 SM_Lek1 P1D2
Initial identification of anchors takes  18.80779504776001
Finding the best anchors took 637.5696477890015
Applying homography to the best anchors takes  156.46743607521057
Initial identification of anchors takes  20.224579095840454
Finding the best anchors took 645.2374651432037
Applying homography to the best anchors takes  157.8263430595398
20230302 SM_Lek1 P2D3
Initial identification of anchors takes  14.759687900543213
Finding the best anchors took 119.84810996055603
Applying homography to the best anchors takes  200.67274117469788
Initial identification of anchors takes  17.438215970993042
Findi